# Now just using DF directly
## no longer need to parse separately into lists etc.  all done internally
## TO Do the dump load and initialize the DB, need to make sure correct order pf tables is loaded first those with no dependencies and then increasingly so.
## now only erroring on original 3 .txt noted by Ian Gorton in original commnication
- - 1. Conf
- - 2. Pub , Keys, Authors, Affiliations (use sets to create unique ones but may change to all .unique)
- - 3. Composites (Many to Many)
- - 4. Paper/abstracts



In [1]:
import sqlite3
import pandas as pd

In [2]:
import sqlcommands as cmd
cmd = reload(cmd)

(1548, 7)
(1548, 7)


In [3]:
df = cmd.createTOTALTable()
df.iloc[50:60]

sqlcommands.py:89: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=None, dtype=None)


,Abstract,Author affiliation,Authors,Conf,Title,terms,year
50,The past 20 years has seen significant investm...,"IBM Software Group, Raleigh, NC, United States...","['Brown, Alan W', '', '', ' McDermid, John A',...",ECSA,The art and science of software architecture,"['Failure analysis', 'Systems engineering', 'O...",2007
51,"Nowadays, Wireless Sensor Networks WSN are a...",Divisi n de Sistemas e Ingenier a Electr nica ...,"['Losilla, Fernando', '', '', 'Vicente Chicote...",ECSA,Wireless sensor network application developmen...,"['Mathematical models', 'Software architecture...",2007
52,Automated services help enterprises create new...,"INFOLAB, Tilburg University, Dept of Informat...","['Papazoglou, Michael P', '', '']",ECSA,What s in a service,"['Condition monitoring', 'Electronic commerce'...",2007
53,The term co evolution describes the symbiotic ...,"University of St Andrews, St Andrews, KY16 9SX...","['Morrison, Ron', '', '', 'Balasubramaniam, Dh...",ECSA,An active architecture approach to dynamic sys...,"['Dynamical systems', 'Grid computing', 'Senso...",2007
54,Software architecture views represent the basi...,Department of Information Systems and Computat...,"['Cordero, Rogelio Limon', '', '', 'Salavert, ...",ECSA,Analyzing styles of the modular software archi...,"['Computer aided design', 'Requirements engine...",2007
55,"Currently, most software systems have a dynami...",Department of Information Systems and Computat...,"['Costa, Crist bal', '', '', 'Ali, Nour', '', ...",ECSA,Dynamic reconfiguration of software architectu...,"['Computer aided design', 'Object oriented pro...",2007
56,We propose a pattern based software developmen...,Department of Computational and Cognitive Scie...,"['C t , Isabelle', '', '', 'Heisel, Marina', '...",ECSA,Pattern based evolution of software architectures,"['Pattern recognition', 'Requirements engineer...",2007
57,Most of the research in the area of multimodal...,"Department of Computer Science, Technische Uni...","['Pereira, Alessandro Costa', ' ,', '', ' Hart...",ECSA,A distributed staged architecture for multimod...,"['Computer programming languages', 'Distribute...",2007
58,The vast diversity of implementation and suppo...,"Computer Languages and Systems II Department, ...","['L pez Sanz, Marcos', '', '', 'Acu a, C sar J...",ECSA,UML profile for the platform independent model...,"['Computer simulation', 'Computer software por...",2007
59,No abstract available,None,"['Dauvin, Jean Claude', ' Romana, Louis Alexan...",ECSA,Hydrobiologia Preface,[nan],2007


In [4]:
#don't need 
#df1 = cmd.getPatentDataFrame(hdffile='../DataBaseParsing/DFstore4.h5') 

In [5]:
df.iloc[0]['Author affiliation']

u'DISP, Univ  of Rome, Rome, Italy'

# Create the Conf Table

In [6]:
confDF = cmd.createConfTable(df, db = 'test3.db')

Opened test3.db database successfully
Created DataFrame
Records created successfully


In [7]:
confDF.to_csv('Tables_v1/conf.csv', sheet_name = 'conf')

# Create publications Table
## Now can parse all the orginal .txt with exception of the same 3  Ian gorton noted were errant

In [8]:
pubDF = cmd.createPublicationsTable(df)

Opened Abstracts_DB.db database successfully
table dropped
Created PUBLICATIONS table
Created DataFrame
Records created successfully


In [9]:
pubDF
pubDF.to_csv('Tables_v1/publications.csv', sheet_name = 'publications')

# Create Keys Table

## There are errors here and I have spent way to ong trying to figure if out.  
## Somewhere the abstracts are being split into the terms.

In [10]:
keys = cmd.createKEYSTable(df)

Opened Abstracts_DB.db database successfully
              keyword
0     Loose couplings
1   aspectual concept
2       Discrete time
3  Polyacrylonitriles
4     Tacit knowledge
Created DataFrame
Records created successfully


In [11]:
print len(keys)
print len(keys.keyword.unique())
keys.iloc[50:75]
keys.to_csv('Tables_v1/keys.csv', sheet_name = 'Keys')

8399
8399


# Authors
## I don't know how to parse them better!  Each text file seems to look at them differently and no idea what to do

In [12]:
authorsDF = cmd.createAUTHORSTable(df)

Opened Abstracts_DB.db database successfully
Author set created
Records created successfully


In [13]:
print len(authorsDF.AuthorName.drop_duplicates())  #<<--- double check!  will keep one 'empty ' ' becuase of set)
authorsDF.head()
authorsDF.to_csv('Tables_v1/authorss.csv', sheet_name = 'authors')

2911


# Affiliation Table
## simple no extra parsing do to differences between entries

In [14]:
affilDF= cmd.createAFFILIATIONTable(df)

Opened Abstracts_DB.db database successfully
Created DataFrame
Records created successfully


In [15]:
affilDF.head()
affilDF.to_csv('Tables_v1/affiliations.csv', sheet_name = 'affilations')

# PAPER TABLE
## should this actually be made by first doing a join or something with the pub table?

In [16]:
paperDF= cmd.createPAPERTable(df)

Opened Abstracts_DB.db database successfully
table dropped
Created PAPER table
Created DataFrame
Records created successfully


In [17]:
paperDF.head()
paperDF.to_csv('Tables_v1/paper.csv', sheet_name = 'paper')

# PAPERKEY COMPOSITE

In [18]:
pk = cmd.createPAPERKEYTable(paperDF)

Opened Abstracts_DB.db database successfully
Records created successfully


sqlcommands.py:558: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  parsedPaper['NEWTERMS'] = parsedPaper[DFCol[-1]].apply(lambda x:


In [19]:
pk.head()
pk.to_csv('Tables_v1/paperKey.csv', sheet_name = 'paperKey')

In [20]:
pk.tail()

,paperID,keyword
25672,1486,u'Carbon corrosion'
25673,1486,u'Chemical Products Generally'
25674,1486,u'Optical Devices and Systems'
25675,1486,u'Fuel cells'
25676,1486,u'Transmission electron microscopy'


In [21]:
pk.merge(paperDF[['paperID', 'confName']], on = 'paperID').head(15)

,paperID,keyword,confName
0,1,u'Research challenges',ECSA
1,1,u'Software product line architecture',ECSA
2,1,u'Materials Science',ECSA
3,1,u'Structural properties',ECSA
4,1,u'Software product lines',ECSA
5,1,u'Software systems',ECSA
6,1,u'Software design',ECSA
7,1,u'Product-lines',ECSA
8,1,u'Computer Programming',ECSA
9,1,u'Feature models',ECSA


# Affiliation_Paper Composite 
## necessary?

In [22]:
paperAffiliationDF = cmd.createAFFILIATIONPAPERTable(paperDF, affilDF)

Opened Abstracts_DB.db database successfully
Records created successfully


In [23]:
paperAffiliationDF.tail()
paperAffiliationDF.to_csv('Tables_v1/paperAffiliation.csv', sheet_name = 'paperAffil')

In [24]:
paperDF[['paperID', 'pubYear']].merge(paperAffiliationDF, on = 'paperID').tail()

,paperID,pubYear,affilID
1481,1482,2014,1107
1482,1483,2012,379
1483,1484,2012,379
1484,1485,2005,971
1485,1486,2012,412


# PAPERAUTHORS COMPOSITE

In [25]:
authorPaper = cmd.createPAPERAUTHORTable(paperDF[['paperID', 'authors']])
authorPaper.to_csv('Tables_v1/authorPaper.csv', sheet_name = 'paperAuthor')

Opened Abstracts_DB.db database successfully
Records created successfully


In [28]:
authorPaper.head(20)

,paperID,authorName
0,1,"Lopez Herrejon, Roberto E"
1,1,"Egyed, Alexander"
2,2,"Perovich, Daniel"
3,2,"Rossel, Pedro O"
4,2,"Bastarrica, Mar a Cecilia"
5,3,"Perovich, Daniel"
6,3,"Rossel, Pedro O"
7,3,"Bastarrica, Mar a Cecilia"
8,4,"Perovich, Daniel"
9,4,"Rossel, Pedro O"


In [29]:
paperDF[['paperID', 'title']].merge(authorPaper, on = 'paperID').head(20)

,paperID,title,authorName
0,1,On the need of safe software product line arch...,"Lopez Herrejon, Roberto E"
1,1,On the need of safe software product line arch...,"Egyed, Alexander"
2,2,Feature model to product architectures Applyi...,"Perovich, Daniel"
3,2,Feature model to product architectures Applyi...,"Rossel, Pedro O"
4,2,Feature model to product architectures Applyi...,"Bastarrica, Mar a Cecilia"
5,3,Feature model to product architectures Applyi...,"Perovich, Daniel"
6,3,Feature model to product architectures Applyi...,"Rossel, Pedro O"
7,3,Feature model to product architectures Applyi...,"Bastarrica, Mar a Cecilia"
8,4,Feature model to product architectures Applyi...,"Perovich, Daniel"
9,4,Feature model to product architectures Applyi...,"Rossel, Pedro O"
